In [3]:
import pandas as pd
import numpy as np

gene_exp = pd.read_csv('/data/keras_autoencoder_imputation/E-GEOD-78193.pcl',
                       delimiter='\t')
print(gene_exp.shape)
print(gene_exp.head())

(16240, 126)
    Gene  GSM2069305  GSM2069306  GSM2069307  GSM2069308  GSM2069309  \
0      1    0.316824    0.252532    0.243955    0.234106    0.292485   
1     10         NaN   -0.138681   -0.131388   -0.154578   -0.133985   
2    100    2.258508    2.220878    2.292011    2.221837    2.408065   
3   1000    0.015773    0.002357   -0.066824   -0.055737    0.070111   
4  10000    0.083632    0.072687    0.073163   -0.005703    0.042765   

   GSM2069310  GSM2069311  GSM2069312  GSM2069313     ...      GSM2069420  \
0    0.268240    0.321052    0.270014    0.269687     ...        0.372067   
1   -0.073763   -0.108244   -0.101171   -0.075025     ...        0.565097   
2    2.321494    2.398902    2.409410    2.167130     ...        1.911440   
3    0.058310   -0.038059   -0.017265   -0.116453     ...       -0.069461   
4    0.041141    0.025783    0.041102    0.042092     ...        0.224562   

   GSM2069421  GSM2069422  GSM2069423  GSM2069424  GSM2069425  GSM2069426  \
0    0.675189 

In [10]:
from keras.layers import Input, Dense
from keras.models import Model
from keras import backend as K

x_train = gene_exp.as_matrix()[:, 1:]

input_exp = Input(shape=(126,))
encoded = Dense(64, activation='relu')(input_exp)
encoded = Dense(32, activation='relu')(encoded)
decoded = Dense(64, activation='relu')(encoded)
decoded = Dense(126, activation='sigmoid')(decoded)

autoencoder = Model(input_exp, decoded)
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')

autoencoder.fit(x_train, x_train,
                batch_size=256,
                epochs=100,
                shuffle=True)

TypeError: fit() got an unexpected keyword argument 'epochs'